In [ ]:
from pathlib import Path

# from tokenizers import ByteLevelBPETokenizer

paths = [str(x) for x in Path("/mnt/data/arxiv/source/tex2txt/before2010").glob("**/*.txt")]


In [ ]:
# read the all the file names in the folder
# folder name: /mnt/data/arxiv/source/tex2txt
import os
import pandas as pd
from tqdm import tqdm
file_names = os.listdir('/mnt/data/arxiv/source/tex2txt/before2010')
# merge all file into a txt
with open('/mnt/data/arxiv/source/before2010.txt', 'w') as f:
    for file_name in tqdm(file_names):
        # read the file
        with open('/mnt/data/arxiv/source/tex2txt/before2010/' + file_name, 'r') as f1:
            text = f1.read()
            f.write(text)
            f.write("\n")

In [ ]:
from tokenizers.processors import BertProcessing


from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('/home/jjdong/wholewordbert')

In [ ]:
tokenizer.encode("This is America")

In [ ]:
tokenizer.encode("This is bioinformatics")


In [ ]:
!nvidia-smi

In [ ]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=53_100,
    max_position_embeddings=258,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)
from transformers import BertTokenizerFast



In [5]:
import multiprocessing
from datasets import load_dataset
# Load the pretrained tokenizer
tokenizer = BertTokenizerFast.from_pretrained("/home/jjdong/wholewordbert", truncation=True, max_len=256)

from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config=config)
tokenizer.model_max_length = 256
tokenizer.model_max_length 
dataset = load_dataset('text', data_files='/mnt/data/arxiv/source/before2010.txt', cache_dir='/media/sdb/datasetcache')

# # Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=256)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 228000/83218786 [00:09<48:58, 28246.03 examples/s]  

In [ ]:
# tokenized_dataset.save_to_disk('/media/sdb/datasetcache')

In [ ]:
from datasets import  load_from_disk
# Load the saved dataset from disk
tokenized_dataset = load_from_disk('/media/sdb/datasetcache')

In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import load_metric


data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

training_args = TrainingArguments(
    output_dir="/home/jjdong/wholewordbert",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=128,
    save_steps=100,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset['train'],  # use the entire tokenized dataset
)

In [ ]:

import warnings

# Ignore all warnings
warnings.filterwarnings('ignore')

trainer.train()